In [1]:
import pandas as pd
mu = 0.407

In [2]:
with open('./min_flux_violation.flux_noEnzRxns.txt') as f:
    rxns = f.read().split('\n')
rxns = [i for i in rxns if i != '']
rxns = [i.split('\t')[0] for i in rxns]

In [3]:
df_enz = pd.read_excel('../scRBA_build_GAMS_model/input/ENZYME_stoich_curation_2021-10-01.xlsx')
df_enz.index = df_enz.id.to_list()

In [4]:
df_prot = pd.read_excel('../scRBA_build_GAMS_model/input/PROTEIN_stoich_curation.xlsx')
df_prot.index = df_prot.id.to_list()

In [5]:
df_data = pd.read_excel('./Bjorkeroth_Nielsen_2020_data.xlsx')
df_data.index = df_data.id.to_list()
cmin = df_data['conc (g/gDW)'].min()

In [6]:
for i in rxns:
    prots = df_enz.protein_stoich[i].split(',')
    prots = [j.split(':')[0] for j in prots]
    
    for p in prots:
        df_data.loc[p, 'id'] = p
        df_data.loc[p, 'name'] = df_prot.name[p]
        df_data.loc[p, 'uniprot'] = df_prot.uniprot[p]
        df_data.loc[p, 'MW (g/mmol)'] = df_prot.loc[p, 'MW (g/mmol)']
        df_data.loc[p, 'type'] = 'gapfill_rbacalc'
        df_data.loc[p, 'conc (g/gDW)'] = cmin
        df_data.loc[p, 'vtrans (mmol/gDW/h)'] = cmin * mu / df_data.loc[p, 'MW (g/mmol)']

In [7]:
df_data.to_excel('./Bjorkeroth_Nielsen_2020_data_gapfilled_frompy.xlsx', index=None)